In [ ]:
from Arm_Lib import Arm_Device
import time, json

arm = Arm_Device()
time.sleep(0.2)
print("DOFBOT connected")

# Optionally disable torque so you can manually or with joystick reposition
arm.Arm_serial_set_torque(0)
print("Torque off ï¿½ arm is free to move manually (joystick / by hand)")

def read_pose():
    return [arm.Arm_serial_servo_read(i) for i in range(1, 7)]

coords = []
for i in range(1, 5):
    input(f"Move arm to POINT {i}, then press ENTER ? ")
    pose = read_pose()
    print("Recorded angles:", pose)
    coords.append(pose)

with open("dofbot_pi_poses.json", "w") as f:
    json.dump(coords, f)

print("Saved poses:", coords)

# To replay ï¿½ re-enable torque, then write
arm.Arm_serial_set_torque(1)
time.sleep(0.2)

for p in coords:
    arm.Arm_serial_servo_write6(*p, 1500)  # 1.5s movement
    time.sleep(2)

print("Replay done")